### This file extracts audio files from directories.
1. The audio files are stored in different folders with their respective class name.
2. The functions will extract melspectrogram and mfcc from each audio files
3. For alignment, the audio files are sliced according the `seg size`(in this sample,seg=30).
4. If the segmentation has sample size less than `seg size` but more than half of it, it will be padded with 0s. Else erased.

In [1]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt


In [2]:
np.__version__

'1.21.5'

In [3]:
root_dir = "/home/assa8945/MMC/Dataset/Watkins_full"
# root_dir = "D:\A\Marine Mammal Detection\datasets\AAAI23\\3classes\\audio"
seg = 240
sample_rate = 4000
# save_dir =  "D:\A\Marine Mammal Detection\datasets\\"+"100khz_seg_"+str(seg)+"_mean_padded"
save_dir = "/home/assa8945/MMC/Dataset/features/watkins_full/full_features"
if os.path.exists(save_dir):
    pass
else:
    os.mkdir(save_dir)

In [4]:
label_name={}

In [5]:
for i,value in enumerate(os.listdir(root_dir)):
    label_name[i]=value
label_name

{0: 'Juan Fernandez Fur Seal',
 1: 'sperm whale',
 2: 'Pantropical Spotted Dolphin',
 3: 'Finback Whale',
 4: 'Humpback Whale',
 5: 'Common Dolphin',
 6: 'Long-Finned Pilot Whale',
 7: 'Weddell Seal',
 8: 'Ross Seal',
 9: 'Gray Seal',
 10: 'Steller Sea Lion',
 11: 'Harp Seal',
 12: 'Finless Porpoise',
 13: 'Harbor Porpoise',
 14: 'West Indian Manatee',
 15: 'Southern Right Whale',
 16: 'Bottlenose Dolphin',
 17: 'New Zealand Fur Seal',
 18: 'Dusky Dolphin',
 19: 'Rough-Toothed Dolphin',
 20: 'Ribbon Seal',
 21: 'Short-Finned (Pacific) Pilot Whale',
 22: 'Ringed Seal',
 23: 'Tucuxi Dolphin',
 24: 'Spinner Dolphin',
 25: 'Leopard Seal',
 26: 'Hooded Seal',
 27: 'Northern Right Whale',
 28: "Fraser's Dolphin",
 29: "Grampus, Risso's Dolphin",
 30: 'Blue Whale',
 31: 'Sea Otter',
 32: 'False Killer Whale',
 33: 'Striped Dolphin',
 34: 'White-sided Dolphin',
 35: 'Irawaddy Dolphin',
 36: 'Narwhal',
 37: 'White-beaked Dolphin',
 38: "Dall's Porpoise",
 39: 'Walrus',
 40: 'Killer Whale',
 41:

In [6]:
# keeplist = [1.0, 3.0, 4.0, 40.0, 21.0, 6.0, 2.0, 24.0, 5.0, 16.0, 7.0, 47.0, 33.0]

In [ ]:
keeplist = ['sperm whale','Finback Whale','Humpback Whale','Killer Whale','Short-Finned (Pacific) Pilot Whale','Long-Finned Pilot Whale','Pantropical Spotted Dolphin','Spinner Dolphin','Common Dolphin','Bottlenose Dolphin','Weddell Seal','Bowhead Whale','Striped Dolphin']

In [7]:
# len(keeplist)

13

In [8]:
# import csv
# class_limited = []
# for i in keeplist:
#     class_limited.extend([label_name[i]])
# len(class_limited)

In [10]:
def slicing_and_padding_data_list(data_list,seg=30):
    """_summary_

    Args:
        data_list (_type_): A data list that contains 2D arrays data.
        seg (int, optional): The length of each sliced data. Defaults to 30.
    """
    new_data_list = []
    for data in data_list: # loop through each data in the data list 
        data_length = data.shape[0] # The length of original 2D data sample.
        seg_counter = 0
        while data_length > 0:
            if data_length>=seg:

                new_data_list.append(data[seg_counter:seg_counter + seg,:])
                data_length -= seg
                seg_counter += seg

            elif data_length >= seg/2:  #when the left data_length is less than seg size but large than half of seg size, pad it up to seg size
                pad = np.zeros((seg-data_length,data.shape[1]))
                padded_data = np.vstack((data[seg_counter:,:],pad))
                # print(padded_data.shape)
                new_data_list.append(padded_data)
                break
            else: #when the row number is less than half of the segment , pass the rows.
                break
    return new_data_list


In [11]:
def slicing_and_padding_single_data(data,seg=30):
    """_summary_

    Args:
        data_list (_type_): A data list that contains 2D arrays data.
        seg (int, optional): The length of each sliced data. Defaults to 30.
    """
    new_data_list = []

    data_length = data.shape[0] # The length of original 2D data sample.
    seg_counter = 0
    while data_length > 0:
        if data_length>=seg:

            new_data_list.append(data[seg_counter:seg_counter + seg,:])
            data_length -= seg
            seg_counter += seg

        elif data_length >= seg/2:  #when the left data_length is less than seg size but large than half of seg size, pad it up to seg size
            
           # pad the sample mean
            average = data.mean(axis=0)
            missing_length = seg-data_length
            averages_pads = np.tile(average,(missing_length,1))
            padded_data = np.vstack((data[seg_counter:,:],averages_pads))
            # print(padded_data.shape)

            #pad 0s 
            # pad = np.zeros((seg-data_length,data.shape[1]))
            # padded_data = np.vstack((data[seg_counter:,:],pad))


            #pad last frame
            # last_frame = data[-1,:]
            # missing_length = seg-data_length
            # pad = np.tile(last_frame,(missing_length,1))
            # padded_data = np.vstack((data[seg_counter:,:],pad))



            # print(padded_data.shape)
            new_data_list.append(padded_data)
            break
        else: #when the row number is less than half of the segment , pass the rows.
            break
    return new_data_list


In [12]:
def batch_feature_extraction(dir,sample_rate,seg=seg,):
    """_summary_

    Args:
        dir (_type_): Root folder directory that contains differents classes' folders.

    Returns:
        _type_: spectrogram list, mfcc list
    """
 
    # Save the folders' directories to a list according to the classes.
    
    folder_list = []
    class_list = []
    for root,folders,files in os.walk(dir):
        for folder in folders:
            # if folder not in class_limited:
            #     continue
            folder_list.append(os.path.join(root,folder))
            class_list.append(folder)
    # print(folder_list)

    print(class_list)
    for folder_dir in folder_list:
        
        specs = []
        mfccs = []
        print('going through folder',folder_dir)
        for file in os.listdir(folder_dir):
            
            if file.endswith(".csv"):
                continue

            audio_name = file
            audio_dir = os.path.join(folder_dir,audio_name)
            # print(audio_dir)

            y, sr = librosa.load(audio_dir,sr=sample_rate) #default sample rate 22050
            if len(y)<2048:
                print('The length of audio is less than 2048 samples ',audio_dir)
                continue
            
            # Calculate spectrogram
            spec = librosa.feature.melspectrogram(y=y,sr=sr,n_mels=240)
            spec = librosa.power_to_db(spec)
            spec = spec.T
            # print(spec.shape)
            modified_spec = slicing_and_padding_single_data(spec,seg=seg)
            specs.extend(modified_spec)           
            
            # Calculate mfcc
            mfcc = librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40)
            mfcc = mfcc.T
            # print(mfcc.shape)
            modified_mfcc = slicing_and_padding_single_data(mfcc,seg=seg)
            
            mfccs.extend(modified_mfcc)

#-------------------------------------------------------------------------------------------
# Assert that all the data has correct shape
        assert len(specs)==len(mfccs),AssertionError
        print('There are {} sliced and padded data samples under \n{}\n'.format(len(specs),folder_dir))
        
        for i in specs:
            assert i.shape[0]==seg,print(i.shape)
        for j in mfccs:
            assert j.shape[0]==seg,print(j.shape)
#-------------------------------------------------------------------------------------------


#-------------------------------------------------------------------------------------------
# Save the data into csv files to respective directories
        specs = np.array(specs)
        # print(specs.shape)
        mfccs = np.array(mfccs)
        # print(mfccs.shape)
        reshaped_specs = specs.reshape(specs.shape[0],-1)
        reshaped_mfccs = mfccs.reshape(mfccs.shape[0],-1)

        np.savetxt(os.path.join(folder_dir,'mfccs.csv'), reshaped_mfccs, delimiter=",")
        np.savetxt(os.path.join(folder_dir,'specs.csv'), reshaped_specs, delimiter=",")


batch_feature_extraction(root_dir,sample_rate=sample_rate,seg=seg)

['sperm whale', 'Pantropical Spotted Dolphin', 'Finback Whale', 'Humpback Whale', 'Common Dolphin', 'Long-Finned Pilot Whale', 'Weddell Seal', 'Bottlenose Dolphin', 'Short-Finned (Pacific) Pilot Whale', 'Spinner Dolphin', 'Striped Dolphin', 'Killer Whale', 'Bowhead Whale']
going through folder /home/assa8945/MMC/Dataset/Watkins_full/sperm whale


/home/assa8945/.local/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


In [ ]:
def merge_and_labels(root_dir,seg):
    folders = os.listdir(root_dir)
    labels = []
    counter = 0
    collected_specs = np.empty((1,seg,240))
    collected_mfccs = np.empty((1,seg,40))
    collected_y = np.empty((1,1))
    for folder in folders:
        if folder.endswith('csv'):
            continue 
        # if folder not in class_limited:
        #     continue
        folder_dir = os.path.join(root_dir,folder)
        
        specs = np.loadtxt(os.path.join(folder_dir,"specs.csv"), delimiter=",")
        mfccs = np.loadtxt(os.path.join(folder_dir,"mfccs.csv"), delimiter=",")

        # Restore data to 3D
        specs = specs.reshape(-1,seg,240)
        mfccs = mfccs.reshape(-1,seg,40)
        y = np.full((specs.shape[0],1),counter)

        # Stack them up
        collected_specs = np.vstack((collected_specs,specs))
        collected_mfccs = np.vstack((collected_mfccs,mfccs))
        collected_y = np.vstack((collected_y,y))
        
        labels.append((folder,counter))

        counter += 1

    # Remove the empty sample 0 (created for initialization)    
    collected_specs = np.delete(collected_specs,0,0)
    collected_mfccs = np.delete(collected_mfccs,0,0)
    collected_y = np.delete(collected_y,0,0)


    print(labels) 
    
    print('\nThe overall mfcc data has shape of ',collected_mfccs.shape)

    print('\nThe overall spectrogram data has shape of ',collected_specs.shape)
    print('\nThe labels have shape of ',collected_y.shape)

    return collected_specs,collected_mfccs,collected_y

collected_specs,collected_mfccs,collected_y = merge_and_labels(root_dir,seg)

KeyboardInterrupt: 

In [ ]:
# The 3D arrays are reshaped into 2D arrays so can be save to csv files
reshaped_mfccs=collected_mfccs.reshape(collected_mfccs.shape[0],-1)
reshaped_specs=collected_specs.reshape(collected_specs.shape[0],-1)
print(collected_mfccs.shape)

if os.path.exists(save_dir):
    pass
else:
    os.mkdir(save_dir)

print("saving data to: ",save_dir)
# np.savetxt(os.path.join(save_dir,'Full_mfccs.csv'), reshaped_mfccs, delimiter=",")
# np.savetxt(os.path.join(save_dir,'Full_specs.csv'), reshaped_specs, delimitmmer=",")
np.savez(os.path.join(save_dir,'mfccs.npy.npz'),*collected_mfccs)
np.savez(os.path.join(save_dir,'specs.npy.npz'),*collected_specs)
np.savetxt(os.path.join(save_dir,'labels.csv'), collected_y, delimiter=",")

print('completed')

(330738, 60, 40)
saving data to:  /home/assa8945/MMC/Dataset/features/watkins_full/xx_classes_300limited
completed
